In [297]:
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display, HTML
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

from geopy.geocoders import Nominatim
from shapely.geometry import Point

from geopy.extra.rate_limiter import RateLimiter
import numpy as np  

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim  
import time

In [ ]:
# set working directory
os.chdir('C:/Users/blah/blah/Documents/Data/WI_RAD')

print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\michelle.iolow\OneDrive - crj.org\Documents\Data\WI_RAD


# read in data files

In [ ]:
new_RAD_df = pd.read_excel('RAD_seperatedcat3.xlsx', sheet_name = 'Combined')

# merge dataframes
neighborhood_tracts = gpd.read_file('C:/Users/blah/blah/Data/Misc/dcdneighborhoods/DCDNeighborhoods.shp')
neighborhood_tracts = neighborhood_tracts.rename(columns = {"Neighborho" : "Neighborhood"})

neighborhood_tracts["Neighborhood"] = neighborhood_tracts["Neighborhood"].str.lower()
RAD_df = pd.merge(new_RAD_df, neighborhood_tracts, on = "Neighborhood", how = "outer")

# turn merged dataframe into geopandas df
RAD_df = gpd.GeoDataFrame(RAD_df, geometry = 'geometry')
RAD_df.to_crs(epsg = 26916, inplace = True)

In [300]:
RAD_df.columns

Index(['Neighborhood', 'ped_RAD', 'totalpop_avg', 'white_not_hisp_avg',
       'black_not_hisp_avg', 'white_pedstops', 'black_pedstops',
       'latinx_pedstops', 'other_pedstops', 'white_vehstops', 'black_vehstops',
       'latinx_vehstops', 'other_vehstops', 'black_Part I_vic',
       'white_Part I_vic', 'totalmale_avg', 'total_under5_avg',
       'total10to14_avg', 'total_5to9_avg', 'total_15to19_avg',
       'total_20to24_avg', 'total_25to29_avg', 'total_60to64_avg',
       'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg',
       'total_80to84_avg', 'total_85andolder_avg', 'total_hispanic_avg',
       'AI_AN_Part I_vic', 'Asian_Part I_vic', 'NH_PI_Part I_vic',
       'hispanic_Part I_vic', 'other_Part I_vic', 'percent_blackpop',
       'percent_whitepop', 'total_PartI_vic', 'black_Part II_vic',
       'white_Part II_vic', 'AI_AN_Part II_vic', 'Asian_Part II_vic',
       'NH_PI_Part II_vic', 'hispanic_Part II_vic', 'other_Part II_vic',
       'total_PartII_vic', 'total_Par

# Pedestrian Stops Analysis

## Pedestrian Stops Data and Variable Coding

In [301]:
PedStop_reg_df = pd.read_csv('Pedestrian_SepOff_RegDF.csv')

In [302]:
PedStop_reg_df.columns.tolist()

['top10neighborhood',
 'Neighborhood',
 'searchperformed',
 'pcsearch',
 'incidentarrestsearch',
 'contactdesc1a',
 'stop_justificationa',
 'stop_justificationb',
 'search_justificationa',
 'search_justificationb',
 'uniqueperson',
 'top20neighborhood',
 'call_no',
 'sexcat',
 'racecat',
 'agecat',
 'agecontinuous',
 'young_old',
 'timeofday',
 'morning',
 'afternoon',
 'firstevening',
 'secondevening',
 'district',
 'stoptype',
 'violentstop',
 'violent_weapon',
 'violent_other',
 'propertystop',
 'drugdealingstop',
 'roadsafetystop',
 'equipmentstop',
 'publicdisorderstop',
 'wantedorwarrantstop',
 'trafficstop',
 'possweaponstop',
 'possdrugstop',
 'otherstop',
 'missingstop',
 'violation_typea',
 'searcharrest',
 'searchnofrisk',
 'friskperformed',
 'searchandfrisk',
 'frisktype',
 'friskarrest',
 'contrabandretrieved',
 'stopoutcome',
 'top20officer',
 'top20unit',
 'anysearch',
 'search_nofrisk',
 'friskandarrest',
 'female',
 'white',
 'black',
 'latino',
 'otherrace',
 'old',
 

In [303]:
# columns to recode
PedStop_Cols = ['male', 'young', 'morning', 'afternoon', 'firstevening', 'secondevening', 'violent_stop', 'searchperf', 'pcsearch', 'friskperf',
                      'contrabandrecd', 'inv20officers', 'inv20units', 'noaction', 'citation', 'arrest', 'summons']
PedStop_RecodeCols = ['male', 'young','violent_stop', 'searchperf', 'pcsearch', 'friskperf',
                      'contrabandrecd', 'inv20officers', 'inv20units', 'noaction','arrest']
recode_dict = {"Yes": 1, 'No': 0}

PedStop_reg_df[PedStop_RecodeCols]= PedStop_reg_df[PedStop_RecodeCols].apply(lambda x: x.map(recode_dict))

PedGroupPop = PedStop_reg_df.groupby(['Neighborhood'], dropna = False, as_index = False)[PedStop_Cols].apply(lambda x: x.sum()/len(x))
PedGroupPop = PedGroupPop.add_suffix("_prop")

RAD_reg_df = pd.merge(RAD_df, PedGroupPop, left_on = 'Neighborhood', 
                      right_on = 'Neighborhood_prop',
                      how = 'left')

In [304]:
unemp_by_neigh_peds = PedStop_reg_df[['Neighborhood', 'Employment/Population Ratio']].drop_duplicates()
RAD_reg_df = pd.merge(RAD_reg_df, unemp_by_neigh_peds, on = "Neighborhood", how = "inner")

In [305]:
RAD_reg_df = RAD_reg_df.rename(columns = {'Employment/Population Ratio' : 'Unemployment_rate'})

In [306]:
RAD_reg_df.columns

Index(['Neighborhood', 'ped_RAD', 'totalpop_avg', 'white_not_hisp_avg',
       'black_not_hisp_avg', 'white_pedstops', 'black_pedstops',
       'latinx_pedstops', 'other_pedstops', 'white_vehstops', 'black_vehstops',
       'latinx_vehstops', 'other_vehstops', 'black_Part I_vic',
       'white_Part I_vic', 'totalmale_avg', 'total_under5_avg',
       'total10to14_avg', 'total_5to9_avg', 'total_15to19_avg',
       'total_20to24_avg', 'total_25to29_avg', 'total_60to64_avg',
       'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg',
       'total_80to84_avg', 'total_85andolder_avg', 'total_hispanic_avg',
       'AI_AN_Part I_vic', 'Asian_Part I_vic', 'NH_PI_Part I_vic',
       'hispanic_Part I_vic', 'other_Part I_vic', 'percent_blackpop',
       'percent_whitepop', 'total_PartI_vic', 'black_Part II_vic',
       'white_Part II_vic', 'AI_AN_Part II_vic', 'Asian_Part II_vic',
       'NH_PI_Part II_vic', 'hispanic_Part II_vic', 'other_Part II_vic',
       'total_PartII_vic', 'total_Par

In [307]:
display(HTML(RAD_reg_df.to_html()))

In [308]:
# percent male
RAD_reg_df['percent_male'] = RAD_reg_df['totalmale_avg']/RAD_reg_df['totalpop_avg']

# percent pop under 15
colnames = ['total_under5_avg', 'total_5to9_avg', 'total10to14_avg']
RAD_reg_df['percent_under15'] = RAD_reg_df[colnames].sum(axis = 1) / RAD_reg_df['totalpop_avg']

# percent pop 15-29
colnames = ['total_15to19_avg', 'total_20to24_avg', 'total_25to29_avg']
RAD_reg_df['percent_1529'] = RAD_reg_df[colnames].sum(axis = 1) / RAD_reg_df['totalpop_avg']

# percent pop over 60
colnames = ['total_60to64_avg', 'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg', 'total_80to84_avg', 'total_85andolder_avg']
RAD_reg_df['percent_over60'] = RAD_reg_df[colnames].sum(axis = 1) / RAD_reg_df['totalpop_avg']
#percent hispanic
RAD_reg_df['percent_hispanic'] = RAD_reg_df['total_hispanic_avg'] / RAD_reg_df['totalpop_avg']

In [309]:
RAD_reg_df['allrace_pedstops'] = RAD_reg_df['white_pedstops'] + RAD_reg_df['black_pedstops'] + RAD_reg_df['latinx_pedstops'] + RAD_reg_df['other_pedstops']

RAD_reg_df['black_ped_allstoprates'] = RAD_reg_df['black_pedstops'] / RAD_reg_df['allrace_pedstops']
RAD_reg_df['white_ped_allstoprates'] = RAD_reg_df['white_pedstops'] / RAD_reg_df['allrace_pedstops']

In [310]:
#fit regression model with stop variables
ped_results_stoponly = smf.ols('black_ped_stoprate ~ male_prop + young_prop + morning_prop + afternoon_prop + secondevening_prop + violent_stop_prop + searchperf_prop + pcsearch_prop + friskperf_prop + contrabandrecd_prop + inv20officers_prop + inv20units_prop + citation_prop + arrest_prop + summons_prop', data = RAD_reg_df).fit()
ped_regsummary_stoponly = ped_results_stoponly.summary()

ped_results_as_html = ped_regsummary_stoponly.tables[1].as_html()
ped_regoutput_df_stoponly = pd.read_html(ped_results_as_html, header=0, index_col=0)[0]

ped_regoutput_df_stoponly.to_csv('multireg_ped_stoponly.csv', index = True)

C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_22624\3066556708.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ped_regoutput_df_stoponly = pd.read_html(ped_results_as_html, header=0, index_col=0)[0]


In [311]:
homicide_rates = pd.read_csv('homicide_rates_only.csv')

# merge in the homicide rates
RAD_reg_df = pd.merge(RAD_reg_df, homicide_rates, how = 'inner', on = 'Neighborhood')

RAD_reg_df = RAD_reg_df.rename(columns = {'total_homicide' : 'total_homicide_vic'})

In [312]:
#fit regression model with stop variables
ped2_results_stoponly = smf.ols('black_ped_allstoprates ~ male_prop + young_prop + morning_prop + secondevening_prop + violent_stop_prop + searchperf_prop + pcsearch_prop + friskperf_prop + contrabandrecd_prop + inv20officers_prop + inv20units_prop + citation_prop + arrest_prop + summons_prop + totalpop_avg + percent_male + percent_1529 + percent_blackpop + percent_hispanic + Unemployment_rate + white_vicrate + black_vicrate + total_homicide_vic', data = RAD_reg_df).fit()
ped2_regsummary_stoponly = ped2_results_stoponly.summary()

ped2_results_as_html = ped2_regsummary_stoponly.tables[1].as_html()
ped2_regoutput_df_stoponly = pd.read_html(ped2_results_as_html, header=0, index_col=0)[0]

ped2_regoutput_df_stoponly.to_csv('multireg_ped_with_demos.csv', index = True)

C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_22624\1159206216.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ped2_regoutput_df_stoponly = pd.read_html(ped2_results_as_html, header=0, index_col=0)[0]


In [259]:
RAD_reg_df.to_csv('regressiondata_peds.csv', index = False)

# Vehicle Regressions

In [261]:
VehStop_reg_df = pd.read_csv('Vehicle_SepOff_RegDF.csv')

C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_22624\2848932215.py:1: DtypeWarning: Columns (1,3,4,29,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  VehStop_reg_df = pd.read_csv('Vehicle_SepOff_RegDF.csv')


In [262]:
VehStop_reg_df.columns.tolist()

['anysearchperformed',
 'contrabandtype_retrieved',
 'uniqueperson',
 'anypcsearch',
 'call_no',
 'TS_reason',
 'stoptype',
 'reckless',
 'wronglane',
 'trafficlight',
 'stopsign',
 'disregardsignal',
 'registration',
 'otherequipment',
 'Neighborhood',
 'top20neighborhood',
 'searchnofrisk',
 'searchperformed',
 'searcharrest',
 'encounterdate',
 'sexcat',
 'racecat',
 'morning',
 'afternoon',
 'firstevening',
 'secondevening',
 'district',
 'friskperformed',
 'searchandfrisk',
 'frisktype',
 'friskarrest',
 'top20officer',
 'top20unit',
 'youngstop',
 'blackstop',
 'latinxstop',
 'whitestop',
 'malestop',
 'male',
 'female',
 'white',
 'black',
 'latino',
 'otherrace',
 'blackwhite',
 'latinowhite',
 'young',
 'old',
 'top10neighborhood',
 'speeding',
 'otherroadsafety',
 'equipmentreg',
 'otherstopreason',
 'searchperf',
 'friskperf',
 'forciblefrisk',
 'contrabandrecd',
 'inv20officers',
 'inv20units',
 'stopresult',
 'noaction',
 'verbalwarn',
 'writtenwarn',
 'citation',
 'arrest

In [ ]:
new_RAD_df = pd.read_excel('RAD_seperatedcat3.xlsx', sheet_name = 'Combined')

# merge dataframes
neighborhood_tracts = gpd.read_file('C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/DCDNeighborhoods.shp')
neighborhood_tracts = neighborhood_tracts.rename(columns = {"Neighborho" : "Neighborhood"})

neighborhood_tracts["Neighborhood"] = neighborhood_tracts["Neighborhood"].str.lower()
RAD_df = pd.merge(new_RAD_df, neighborhood_tracts, on = "Neighborhood", how = "outer")

# turn merged dataframe into geopandas df
RAD_df = gpd.GeoDataFrame(RAD_df, geometry = 'geometry')
RAD_df.to_crs(epsg = 26916, inplace = True)

In [264]:
RAD_df.columns.tolist()

['Neighborhood',
 'ped_RAD',
 'totalpop_avg',
 'white_not_hisp_avg',
 'black_not_hisp_avg',
 'white_pedstops',
 'black_pedstops',
 'latinx_pedstops',
 'other_pedstops',
 'white_vehstops',
 'black_vehstops',
 'latinx_vehstops',
 'other_vehstops',
 'black_Part I_vic',
 'white_Part I_vic',
 'totalmale_avg',
 'total_under5_avg',
 'total10to14_avg',
 'total_5to9_avg',
 'total_15to19_avg',
 'total_20to24_avg',
 'total_25to29_avg',
 'total_60to64_avg',
 'total_65to69_avg',
 'total_70to74_avg',
 'total_75to79_avg',
 'total_80to84_avg',
 'total_85andolder_avg',
 'total_hispanic_avg',
 'AI_AN_Part I_vic',
 'Asian_Part I_vic',
 'NH_PI_Part I_vic',
 'hispanic_Part I_vic',
 'other_Part I_vic',
 'percent_blackpop',
 'percent_whitepop',
 'total_PartI_vic',
 'black_Part II_vic',
 'white_Part II_vic',
 'AI_AN_Part II_vic',
 'Asian_Part II_vic',
 'NH_PI_Part II_vic',
 'hispanic_Part II_vic',
 'other_Part II_vic',
 'total_PartII_vic',
 'total_PartI_II_combined',
 'black_total_vic',
 'white_total_vic',
 '

In [265]:
print(len(RAD_df))

190


In [266]:
VehStop_reg_df.columns.tolist()

['anysearchperformed',
 'contrabandtype_retrieved',
 'uniqueperson',
 'anypcsearch',
 'call_no',
 'TS_reason',
 'stoptype',
 'reckless',
 'wronglane',
 'trafficlight',
 'stopsign',
 'disregardsignal',
 'registration',
 'otherequipment',
 'Neighborhood',
 'top20neighborhood',
 'searchnofrisk',
 'searchperformed',
 'searcharrest',
 'encounterdate',
 'sexcat',
 'racecat',
 'morning',
 'afternoon',
 'firstevening',
 'secondevening',
 'district',
 'friskperformed',
 'searchandfrisk',
 'frisktype',
 'friskarrest',
 'top20officer',
 'top20unit',
 'youngstop',
 'blackstop',
 'latinxstop',
 'whitestop',
 'malestop',
 'male',
 'female',
 'white',
 'black',
 'latino',
 'otherrace',
 'blackwhite',
 'latinowhite',
 'young',
 'old',
 'top10neighborhood',
 'speeding',
 'otherroadsafety',
 'equipmentreg',
 'otherstopreason',
 'searchperf',
 'friskperf',
 'forciblefrisk',
 'contrabandrecd',
 'inv20officers',
 'inv20units',
 'stopresult',
 'noaction',
 'verbalwarn',
 'writtenwarn',
 'citation',
 'arrest

In [267]:
print(len(RAD_df))
print(len(VehStop_reg_df))

190
54398


In [268]:
unemp_by_neigh = VehStop_reg_df[['Neighborhood', 'Employment/Population Ratio']].drop_duplicates()
RAD_Veh_reg_df = pd.merge(RAD_df, unemp_by_neigh, on = "Neighborhood", how = "inner")

In [269]:
# columns to recode
VehStop_Cols = ['male', 'young', 'morning', 'afternoon', 'firstevening', 'secondevening', 'speeding', 'otherroadsafety', 'equipmentreg', 'otherstopreason', 'searchperf','anypcsearch', 'friskperf', 'contrabandrecd', 'inv20officers', 'inv20units', 'noaction', 'verbalwarn', 'writtenwarn', 'citation', 'arrest']
VehStop_RecodeCols = ['male', 'young', 'speeding', 'otherroadsafety', 'equipmentreg', 'otherstopreason', 'searchperf','anypcsearch', 'friskperf', 'contrabandrecd', 'inv20officers', 'inv20units']

recode_dict = {"Yes": 1, 'No': 0}

VehStop_reg_df[VehStop_RecodeCols]= VehStop_reg_df[VehStop_RecodeCols].apply(lambda x: x.map(recode_dict))

VehGroupPop = VehStop_reg_df.groupby(['Neighborhood'], dropna = False, as_index = False)[VehStop_Cols].apply(lambda x: x.sum()/len(x))
VehGroupPop = VehGroupPop.add_suffix("_prop")

RAD_Veh_reg_df = pd.merge(RAD_Veh_reg_df, VehGroupPop, left_on = 'Neighborhood', 
                      right_on = 'Neighborhood_prop',
                      how = 'left')

In [270]:
RAD_Veh_reg_df.columns.tolist()

['Neighborhood',
 'ped_RAD',
 'totalpop_avg',
 'white_not_hisp_avg',
 'black_not_hisp_avg',
 'white_pedstops',
 'black_pedstops',
 'latinx_pedstops',
 'other_pedstops',
 'white_vehstops',
 'black_vehstops',
 'latinx_vehstops',
 'other_vehstops',
 'black_Part I_vic',
 'white_Part I_vic',
 'totalmale_avg',
 'total_under5_avg',
 'total10to14_avg',
 'total_5to9_avg',
 'total_15to19_avg',
 'total_20to24_avg',
 'total_25to29_avg',
 'total_60to64_avg',
 'total_65to69_avg',
 'total_70to74_avg',
 'total_75to79_avg',
 'total_80to84_avg',
 'total_85andolder_avg',
 'total_hispanic_avg',
 'AI_AN_Part I_vic',
 'Asian_Part I_vic',
 'NH_PI_Part I_vic',
 'hispanic_Part I_vic',
 'other_Part I_vic',
 'percent_blackpop',
 'percent_whitepop',
 'total_PartI_vic',
 'black_Part II_vic',
 'white_Part II_vic',
 'AI_AN_Part II_vic',
 'Asian_Part II_vic',
 'NH_PI_Part II_vic',
 'hispanic_Part II_vic',
 'other_Part II_vic',
 'total_PartII_vic',
 'total_PartI_II_combined',
 'black_total_vic',
 'white_total_vic',
 '

In [271]:
print(len(RAD_Veh_reg_df))

150


In [272]:
RAD_Veh_reg_df['allrace_pedstops'] = RAD_Veh_reg_df['white_pedstops'] + RAD_Veh_reg_df['black_pedstops'] + RAD_Veh_reg_df['latinx_pedstops'] + RAD_Veh_reg_df['other_pedstops']

RAD_Veh_reg_df['black_ped_allstoprates'] = RAD_Veh_reg_df['black_pedstops'] / RAD_Veh_reg_df['allrace_pedstops']
RAD_Veh_reg_df['white_ped_allstoprates'] = RAD_Veh_reg_df['white_pedstops'] / RAD_Veh_reg_df['allrace_pedstops']

In [273]:
# percent male
RAD_Veh_reg_df['percent_male'] = RAD_Veh_reg_df['totalmale_avg']/RAD_Veh_reg_df['totalpop_avg']

# percent pop under 15
colnames = ['total_under5_avg', 'total_5to9_avg', 'total10to14_avg']
RAD_Veh_reg_df['percent_under15'] = RAD_Veh_reg_df[colnames].sum(axis = 1) / RAD_Veh_reg_df['totalpop_avg']

# percent pop 15-29
colnames = ['total_15to19_avg', 'total_20to24_avg', 'total_25to29_avg']
RAD_Veh_reg_df['percent_1529'] = RAD_Veh_reg_df[colnames].sum(axis = 1) / RAD_Veh_reg_df['totalpop_avg']

# percent pop over 60
colnames = ['total_60to64_avg', 'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg', 'total_80to84_avg', 'total_85andolder_avg']
RAD_Veh_reg_df['percent_over60'] = RAD_Veh_reg_df[colnames].sum(axis = 1) / RAD_Veh_reg_df['totalpop_avg']
#percent hispanic
RAD_Veh_reg_df['percent_hispanic'] = RAD_Veh_reg_df['total_hispanic_avg'] / RAD_Veh_reg_df['totalpop_avg']

In [274]:
homicide_rates = pd.read_csv('homicide_rates_only.csv')

# merge in the homicide rates
RAD_Veh_reg_df = pd.merge(RAD_Veh_reg_df, homicide_rates, how = 'inner', on = 'Neighborhood')

RAD_Veh_reg_df = RAD_Veh_reg_df.rename(columns = {'total_homicide' : 'total_homicide_vic'})

In [275]:
print(len(RAD_Veh_reg_df))

150


In [276]:
#fit regression model with stop variables
veh_results_stoponly = smf.ols('black_ped_allstoprates ~ male_prop + young_prop + morning_prop + afternoon_prop + secondevening_prop + otherroadsafety_prop + equipmentreg_prop + otherstopreason_prop + searchperf_prop +anypcsearch_prop + friskperf_prop + contrabandrecd_prop + inv20officers_prop + inv20units_prop + verbalwarn_prop + writtenwarn_prop + citation_prop + arrest_prop', data = RAD_Veh_reg_df).fit()
veh_regsummary_stoponly = veh_results_stoponly.summary()

veh_results_as_html = veh_regsummary_stoponly.tables[1].as_html()
veh_regoutput_df_stoponly = pd.read_html(veh_results_as_html, header=0, index_col=0)[0]

veh_regoutput_df_stoponly.to_csv('multireg_veh_stoponly.csv', index = True)

C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_22624\926831145.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  veh_regoutput_df_stoponly = pd.read_html(veh_results_as_html, header=0, index_col=0)[0]


In [277]:
RAD_Veh_reg_df = RAD_Veh_reg_df.rename(columns = {'Employment/Population Ratio' : 'UnemploymentRate'})

In [278]:
#including the demographic variables
results = smf.ols('black_ped_allstoprates ~ male_prop + young_prop + morning_prop + afternoon_prop + secondevening_prop + otherroadsafety_prop + equipmentreg_prop + otherstopreason_prop + searchperf_prop +anypcsearch_prop + friskperf_prop + contrabandrecd_prop + inv20officers_prop + inv20units_prop + verbalwarn_prop + writtenwarn_prop + citation_prop + arrest_prop + totalpop_avg + totalmale_avg + percent_1529 + percent_blackpop + percent_hispanic + UnemploymentRate+ white_vicrate + black_vicrate + total_homicide_vic', data = RAD_Veh_reg_df).fit()
regsummary = results.summary()

results_as_html = regsummary.tables[1].as_html()
regoutput_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

regoutput_df.to_csv('multireg_veh_with_demos.csv', index = True)

C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_22624\1372331081.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  regoutput_df = pd.read_html(results_as_html, header=0, index_col=0)[0]


In [279]:
RAD_Veh_reg_df.to_csv('regressiondata_vehicles.csv', index = False)